In [1]:
# 必要なライブラリをインポートする
from importlib import reload
from helper import run
import bloomfilter
import block
import ecc
import helper
import merkleblock
import network
import script
import tx

In [3]:
from helper import hash256
# ブルームフィルターを作成するソース
bit_field_size = 10
# 入れ物を10個用意する。
bit_field = [0] * bit_field_size
# ハッシュ値を生成する
h = hash256(b'hello world')
#  対象の格納先
bit = int.from_bytes(h, 'big') % bit_field_size
# 該当の格納場所を1にする。
bit_field[bit] = 1
print(bit_field)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [4]:
from helper import hash256
bit_field_size = 10
bit_field = [0] * bit_field_size
# 配列の要素ごとにループを行う
for item in (b'hello world', b'goodbye'):
    h = hash256(item)
    bit = int.from_bytes(h, 'big') % bit_field_size
    # 格納する
    bit_field[bit] = 1
print(bit_field)

[0, 0, 1, 0, 0, 0, 0, 0, 0, 1]


### 練習問題 1

Calculate the Bloom Filter for 'hello world' and 'goodbye' using the `hash160` hash function over a bit field of 10.

In [5]:
# 練習問題 1

from helper import hash160

bit_field_size = 10
bit_field = [0] * bit_field_size
items = (b'hello world', b'goodbye')
# loop through each item
for item in items :
    # hash160 the item
    h = hash160(item)
    # 10でのモジュロを算出する。
    bit = int.from_bytes(h, 'big') % bit_field_size
    # set that bit in bit_field to 1
    bit_field[bit] = 1
# print the bit_field
print(bit_field)

[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]


In [6]:
from helper import hash256, hash160
bit_field_size = 10
bit_field = [0] * bit_field_size
for item in (b'hello world', b'goodbye'):
    for hash_function in (hash256, hash160):
        h = hash_function(item)
        # 10でのモジュロを算出する。
        bit = int.from_bytes(h, 'big') % bit_field_size
        bit_field[bit] = 1
print(bit_field)

[1, 1, 1, 0, 0, 0, 0, 0, 0, 1]


In [7]:
from helper import murmur3
from bloomfilter import BIP37_CONSTANT
field_size = 2
num_functions = 2
tweak = 42
# 入れ物をサイズを16
bit_field_size = field_size * 8
bit_field = [0] * bit_field_size
# 要素ごとにループする
for phrase in (b'hello world', b'goodbye'):
    for i in range(num_functions):
        seed = i * BIP37_CONSTANT + tweak
        h = murmur3(phrase, seed=seed)
        bit = h % bit_field_size
        bit_field[bit] = 1
print(bit_field)

[0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0]


### 練習問題 2

Given a Bloom Filter with size=10, function count=5, tweak=99, what are the bytes that are set after adding these items? (use `bit_field_to_bytes` to convert to bytes)

* `b'Hello World'`
* `b'Goodbye!'`

In [8]:
# 練習問題 2

from bloomfilter import BloomFilter, BIP37_CONSTANT
from helper import bit_field_to_bytes, murmur3

field_size = 10
function_count = 5
tweak = 99
items = (b'Hello World',  b'Goodbye!')
# calculate the bitfield size
bit_field_size = field_size * 8
# create an empty bit field
bit_field = [0] * bit_field_size
# loop through items
for phrase in items :
    # 2つのハッシュ関数ごとにループする
    for i in range(num_functions):
        # calculate the seed
        seed = i * BIP37_CONSTANT + tweak
        # get the murmur3 hash of the item using the seed
        h = murmur3(phrase, seed=seed)
        # mod by the bitfield size
        bit = h % bit_field_size
        # set the bit
        bit_field[bit] = 1
# convert the bit field to bytes
# print the bytes in hex
print(bit_field_to_bytes(bit_field).hex())

4000000a000000000800


### 練習問題 3

Write the `add` method for `BloomFilter`

#### Make [this test](/edit/code-ch12/bloomfilter.py) pass: `bloomfilter.py:BloomFilterTest:test_add`

In [9]:
# 練習問題 3

reload(bloomfilter)
run(bloomfilter.BloomFilterTest("test_add"))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### 練習問題 4

Write the  `filterload` payload from the `BloomFilter` class.

#### Make [this test](/edit/code-ch12/bloomfilter.py) pass: `bloomfilter.py:BloomFilterTest:test_filterload`

In [10]:
# 練習問題 4

reload(bloomfilter)
run(bloomfilter.BloomFilterTest("test_filterload"))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### 練習問題 5

Write the `serialize` method for the `GetDataMessage` class.

#### Make [this test](/edit/code-ch12/network.py) pass: `network.py:GetDataMessageTest:test_serialize`

In [4]:
# 練習問題 5

reload(network)
run(network.GetDataMessageTest("test_serialize"))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [3]:
from bloomfilter import BloomFilter
from helper import decode_base58
from merkleblock import MerkleBlock
from network import FILTERED_BLOCK_DATA_TYPE, GetHeadersMessage, GetDataMessage, HeadersMessage, SimpleNode
from tx import Tx
# 前のトランザクションのハッシュ値
last_block_hex = '00000000000538d5c2246336644f9a4956551afb44ba47278759ec55ea912e19'
address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
h160 = decode_base58(address)
node = SimpleNode('testnet.programmingbitcoin.com', testnet=True, logging=False)
# ブルームフィルターを作成
bf = BloomFilter(size=30, function_count=5, tweak=90210)
bf.add(h160)
node.handshake()
# ブルームフィルターを送信する。
node.send(bf.filterload())
start_block = bytes.fromhex(last_block_hex)
# ブロックヘッダーを取得する。
getheaders = GetHeadersMessage(start_block=start_block)
# ブロックヘッダーを送信する。
node.send(getheaders)
headers = node.wait_for(HeadersMessage)
# getdataメッセージを作成します。
getdata = GetDataMessage()
for b in headers.blocks:
    if not b.check_pow(): # PoWが有効かチェックする。
        raise RuntimeError('proof of work is invalid')
        # 対象トランザクションを含む可能性のあるマークルブロックをリクエストする。
    getdata.add_data(FILTERED_BLOCK_DATA_TYPE, b.hash())
node.send(getdata)
found = False
while not found:
    # 対象トランザクションの抱合を証明するmerkleblockコマンドとTxコマンドを待つ
    message = node.wait_for(MerkleBlock, Tx)
    if message.command == b'merkleblock':
        # トランザクションが抱合されているか確認する。
        if not message.is_valid():
            raise RuntimeError('invalid merkle proof')
    else:
        for i, tx_out in enumerate(message.tx_outs):
            # アドレスが一致していれば出力してループを終了する。
            if tx_out.script_pubkey.address(testnet=True) == address:
                print('found: {}:{}'.format(message.id(), i))
                found = True
                break

found: e3930e1e566ca9b75d53b0eb9acb7607f547e1182d1d22bd4b661cfe18dcddf1:0


### 練習問題 6

Get the current testnet block ID, send yourself some testnet coins, find the UTXO corresponding to the testnet coin _without using a block explorer_, create a transaction using that UTXO as an input and broadcast the `tx` message on the  testnet network.

In [5]:
# 練習問題 6

import time

from block import Block
from bloomfilter import BloomFilter
from ecc import PrivateKey
from helper import hash256, little_endian_to_int, encode_varint, read_varint, decode_base58, SIGHASH_ALL
from merkleblock import MerkleBlock
from network import (
    GetDataMessage,
    GetHeadersMessage,
    HeadersMessage,
    NetworkEnvelope,
    SimpleNode,
    TX_DATA_TYPE,
    FILTERED_BLOCK_DATA_TYPE,
)
from script import p2pkh_script, Script
from tx import Tx, TxIn, TxOut

# 取得し始めるブロックの位置を用意する。
last_block_hex = '00000000000000000000690c48e05683923aba959c22a7b700d9f86f63f487bb'  
# 秘密鍵を生成する。
secret = little_endian_to_int(hash256(b'bloomtest'))  # FILL THIS IN
private_key = PrivateKey(secret=secret)
# アドレスを生成する。
addr = private_key.point.address(testnet=True)
# base58でデコードする
h160 = decode_base58(addr)
# 宛先アドレス
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
target_h160 = decode_base58(target_address)
# Scriptを作成する。
target_script = p2pkh_script(target_h160)
# 手数料
fee = 5000  # fee in satoshis


# connect to testnet.programmingbitcoin.com in testnet mode
node = SimpleNode('testnet.programmingbitcoin.com', testnet=True, logging=False)
# ブルームフィルターを作成
bf = BloomFilter(30, 5, 90210)
# add the h160 to the bloom filter
bf.add(h160)
# 通信を確立する。
node.handshake()
# load the bloom filter with the filterload command
node.send(bf.filterload())

# set start block to last_block from above
start_block = bytes.fromhex(last_block_hex)
# send a getheaders message with the starting block
getheaders = GetHeadersMessage(start_block=start_block)
node.send(getheaders)

# 対象トランザクションを含むヘッダーが送信されるまで待つ
headers = node.wait_for(HeadersMessage)
# store the last block as None
last_block = None
# initialize the GetDataMessage
getdata = GetDataMessage()
# loop through the blocks in the headers
for b in headers.blocks:
    # check that the proof of work on the block is valid
    if not b.check_pow():
        raise RuntimeError('proof of work is invalid')
    # check that this block's prev_block is the last block
    if last_block is not None and b.prev_block != last_block :
        raise RuntimeError('chain broken')
    # add a new item to the get_data_message
    # should be FILTERED_BLOCK_DATA_TYPE and block hash
    getdata.add_data(FILTERED_BLOCK_DATA_TYPE, b.hash())
    # set the last block to the current hash
    last_block = b.hash()
# send the getdata message
node.send(getdata)

# initialize prev_tx and prev_index to None
prev_tx = None
prev_index = None
prev_amount = None
# loop while prev_tx is None 
while prev_tx is None :
    # wait for the merkleblock or tx commands
    message = node.wait_for(MerkleBlock, Tx)
    # if we have the merkleblock command
    if message.command == b'merkleblock' :
        # check that the MerkleBlock is valid
        if not message.is_valid() :
            raise RuntimeError('invalid merkle proof')
    # else we have the tx command
    else :
        # set the tx's testnet to be True
        message.testnet = True
        # loop through the tx outs
        for i, tx_out in enumerate(message.tx_outs) :
            # アドレスが一致していれば出力してループを終了する。
            # if our output has the same address as our address we found it
            if tx_out.script_pubkey.address(testnet=True) == address:     
                # we found our utxo. set prev_tx, prev_index, and tx
                prev_tx = message.hash()
                prev_index = 1
                prev_amount = tx_out.amount
                print('found: {}:{}'.format(prev_tx, prev_index))
# create the TxIn
tx_ins = TxIn(prev_tx, prev_index)
# アウトプットの額を算出する
output_amount = prev_amount - fee
# create a new TxOut to the target script with the output amount
tx_outs = TxOut(output_amount, target_pubkey)
# create a new transaction with the one input and one output
tx_obj = Tx(1, [tx_ins], [tx_outs], 0, testnet=True)
# sign the only input of the transaction
print(tx_obj.sign_input(0, privatekey))
# serialize and hex to see what it looks like
print(tx_obj.serialize().hex())
# send this signed transaction on the network
node.send(tx_obj)
# wait a sec so this message goes through with time.sleep(1) 
time.sleep(1) 
# now ask for this transaction from the other node
# create a GetDataMessage
getdata = GetDataMessage()
# ask for our transaction by adding it to the message
getdata.add_data(FILTERED_BLOCK_DATA_TYPE, b.hash())
# send the message
node.send(getdata)
# now wait for a Tx response
received_tx = node.wait_for(Tx)
# if the received tx has the same id as our tx, we are done!
if received_tx.id() == tx_obj.id() :
    print('success!!')

KeyboardInterrupt: 